# Amazon SageMaker で使用するコンテナイメージを準備する

パイプラインから SageMaker の機能を使ってデータ準備、モデルの学習、バッチ推論を実行する際に使用するコンテナイメージを作成します。このノートブックは、SageMaker ノートブックインスタンスでのみ動作します。SageMaker Studio を使う場合は `docker` コマンドを使えないので代わりに [`sm-docker`](https://aws.amazon.com/jp/blogs/machine-learning/using-the-amazon-sagemaker-studio-image-build-cli-to-build-container-images-from-your-studio-notebooks/) を使ってください。

In [ ]:
import boto3
import pandas as pd
import sagemaker

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

project_name = 'sagemaker-ml-pipeline'
user_name = 'demo'

In [ ]:
def build_and_push_image(repo_name, docker_path, extra_accounts=[], tag = ':latest'):
    uri_suffix = 'amazonaws.com'
    repository_uri = '{}.dkr.ecr.{}.{}/{}'.format(account_id, region, uri_suffix, repo_name + tag)

    for a in extra_accounts:
        !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {a}.dkr.ecr.{region}.amazonaws.com
    !docker build -t $repo_name $docker_path
    !aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com
    !aws ecr create-repository --repository-name $repo_name
    !docker tag {repo_name + tag} $repository_uri
    !docker push $repository_uri
    return repository_uri

## モデル学習用コンテナイメージの作成

In [ ]:
image_name = project_name + '-train-' + user_name
train_repository_uri = build_and_push_image(image_name, './docker/train')

上記セルの出力の最後に Pushed と書かれていれば、ビルドしたコンテナイメージが Amazon ECR に格納されているはずです。以下のセルを実行して表示されたリンクから、latest というタグが付いたイメージがあることを確認してください。

In [ ]:
from IPython.display import display, Markdown
text = f'<a href=\"https://{region}.console.aws.amazon.com/ecr/repositories/private/{account_id}/{image_name}?region={region}\" target=\"_blank\">ECR コンソール</a>'
display(Markdown(text))

## データ準備用コンテナイメージの作成

In [ ]:
image_name = project_name + '-prep-' + user_name
prep_repository_uri = build_and_push_image(image_name, './docker/prep')

text = f'<a href=\"https://{region}.console.aws.amazon.com/ecr/repositories/private/{account_id}/{image_name}?region={region}\" target=\"_blank\">ECR コンソール</a>'
display(Markdown(text))

別のノートブックで使用するため、変数を保存します。

In [ ]:
%store project_name user_name train_repository_uri prep_repository_uri